In [ ]:
# import streamlit as st
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_history_aware_retriever,create_retrieval_chain
# from langchain.vectorstores import Chroma
# from langchain_community.chat_message_histories import ChatMessageHistory
# from langchain_core.chat_history import BaseChatMessageHistory
# from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
# from langchain_groq import ChatGroq
# import os 
# from langchain_community.embeddings import OllamaEmbeddings
# from dotenv import load_dotenv
# from langchain_community.document_loaders import PyPDFLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_core.runnables.history import RunnableWithMessageHistory
# load_dotenv()
# groqapikey=os.getenv("GROQ_API_KEY")
# embeddings=OllamaEmbeddings(model="gemma:2b")
# st.title("Conversational RAG With PDF Upload and Chat Hisory ")
# st.write("Conversational RAG with PDF Uploads and chat history")
# st.write("Uplaod PDF and chat with their content")
# api_key=st.text_input("Enter your Groq API Key:",type="password")
# if api_key:
#     llm=ChatGroq(model="Gemma2-9b-It",api_key=groqapikey)
#     session_id=st.text_input("Session ID",value="default_session")
#     if 'store' not in st.session_state:
#         st.sesion_state.store={}
#     uploaded_files=st.file_uploader("choose a PDF File", type="pdf",accept_multiple_files=True)
#     if uploaded_files:
#         documents=[]
#         for uploaded_file in uploaded_files:
#             temppdf=f"./temp.pdf"
#             with open(temppdf,"wb")as file:
#                 file.write(uploaded_file.getvalue())
#                 file_name=uploaded_file.name
#             loader=PyPDFLoader(temppdf)
#             docs=loader.load()
#             documents.extend(docs)
#         textsplitter=RecursiveCharacterTextSplitter(chunk_sixe=500,chunk_overlap=50)
#         splits=textsplitter.split_documents(documents=documents)
#         vectorstore=Chroma.from_documents(splits,embedding=embeddings)
#         retriever=vectorstore.as_retriever()
#         contextualize_q_system_prompt=(
#         "Given a chat history and the latest user question",
#         "which might be reference context in the chat history",
#         "formulate a standalone question which can be understood",
#         "without the chat history.Do not answer teh question",
#         "just reformulate it if needed and otherwise return it as is"
#     )
#         contextualize_q_prompt=ChatPromptTemplate.from_messages(
#         [
#             ("system",contextualize_q_system_prompt),
#             MessagesPlaceholder("chat_history"),
#             ("human","{input}")
#         ]
#     )
#         history_aware_retrieever=create_history_aware_retriever(llm,retriever,contextualize_q_prompt)
#         system_prompt=(
#         "you are an assistant for question-answering task"
#         "Use the following pieces of retrieved context to answer"
#         "the question. If you dont know the answer ,say that you"
#         "dont know .Use three sentences maximum and keep the answer concise"
#         "\n\n"
#         "{context}"
#     )
#         qa_prompt=ChatPromptTemplate.from_messages(
#         [
#             "system",system_prompt,
#             MessagesPlaceholder("chat_history"),
#             ("human","{input}")
#         ]
#         )
#         question_answerchain=create_stuff_documents_chain(llm,qa_prompt)
#         ragchain=create_retrieval_chain(history_aware_retrieever,question_answerchain)
#         def get_session_history(session_id:str)->BaseChatMessageHistory:
#           if session_id not in st.session_state.store:
#             st.session_state.store[session_id]=ChatMessageHistory()

#           return st.session_state.store[session_id]
#         conversational_rag_chain=RunnableWithMessageHistory(
#         ragchain,
#         get_session_history=get_session_history,
#         input_messages_key="input",
#         history_messages_key="chathistory",
#         output_messages_key="answer"
#         )

    
        
import streamlit as st
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_community.embeddings import OllamaEmbeddings
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory
import os

load_dotenv()
groqapikey = os.getenv("GROQ_API_KEY")

embeddings = OllamaEmbeddings(model="gemma:2b")

st.title("Conversational RAG With PDF Upload and Chat History")
st.write("Conversational RAG with PDF Uploads and chat history")
st.write("Upload PDF and chat with their content")

api_key = st.text_input("Enter your Groq API Key:", type="password")

if api_key:
    llm = ChatGroq(model="Gemma2-9b-It", api_key=groqapikey)
    session_id = st.text_input("Session ID", value="default_session")

    
    if 'store' not in st.session_state:
        st.session_state.store = {}

    uploaded_files = st.file_uploader("Choose a PDF File", type="pdf", accept_multiple_files=True)

    if uploaded_files:
        documents = []

        for uploaded_file in uploaded_files:
            temppdf = "./temp.pdf"
            with open(temppdf, "wb") as file:
                file.write(uploaded_file.getvalue())
            loader = PyPDFLoader(temppdf)
            docs = loader.load()
            documents.extend(docs)

        # Fix: typo in `chunk_size`
        textsplitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        splits = textsplitter.split_documents(documents=documents)

        vectorstore = Chroma.from_documents(splits, embedding=embeddings)
        retriever = vectorstore.as_retriever()

        contextualize_q_system_prompt = (
            "Given a chat history and the latest user question, "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do not answer the question, "
            "just reformulate it if needed and otherwise return it as is."
        )

        contextualize_q_prompt = ChatPromptTemplate.from_messages([
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ])

        history_aware_retriever = create_history_aware_retriever(
            llm, retriever, contextualize_q_prompt
        )

        system_prompt = (
            "You are an assistant for a question-answering task. "
            "Use the following pieces of retrieved context to answer "
            "the question. If you don't know the answer, say that you "
            "don't know. Use three sentences maximum and keep the answer concise.\n\n"
            "{context}"
        )

        qa_prompt = ChatPromptTemplate.from_messages([
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}")
        ])

        question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
        rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

        def get_session_history(session_id: str) -> BaseChatMessageHistory:
            if session_id not in st.session_state.store:
                st.session_state.store[session_id] = ChatMessageHistory()
            return st.session_state.store[session_id]

        conversational_rag_chain = RunnableWithMessageHistory(
            rag_chain,
            get_session_history=get_session_history,
            input_messages_key="input",
            history_messages_key="chat_history",
            output_messages_key="answer"
        )
        user_input=st.text_input('Your question')
        if user_input:
            session_history=get_session_history(session_id=session_id)
            response=conversational_rag_chain.invoke(
                
                {"input":user_input},
                config={
                    "configurable":{"session_id":session_id}
                }
            )

            st.write(st.session_state.store)
            # st.success("Assistant:",response["answer"])
            # st.success(f"Assistant: {response['answer']}")
            st.success(f"Assistant: {response['answer']}") # Correct


            st.write("Chat History",session_history.messages)
else:
    st.warning("Please enter the groq API key")



        





















































